In [103]:
def read_console(path):
    with open(path, 'r') as f:
        console = [l.replace('\n', '') for l in f.readlines()]
    return console

class Directory():
    def __init__(self, name, level, parent):
        self.name = name
        self.level = level
        self.files = []
        self.parent = parent
        self.subdirectories = []

    def get_size(self):
        size = 0
        for file in self.files:
            size += file.get_size()
        for subdirectory in self.subdirectories:
            size += subdirectory.get_size()
        return size

    def access_subdirectory(self, directory_name):
        for subdir in self.subdirectories:
            if subdir.name == directory_name:
                return subdir
        new_subdirectory = self.add_subdirectory(directory_name)
        return new_subdirectory

    def add_subdirectory(self, directory_name):
        if self.get_subdirectory(directory_name):
            return self.get_subdirectory(directory_name)
        else:
            new_subdirectory = Directory(directory_name, self.level+1, self)
            self.subdirectories.append(new_subdirectory)
            return new_subdirectory

    def get_subdirectory(self, directory_name):
        for subdir in self.subdirectories:
            if subdir.name == directory_name:
                return subdir
        return None

    def add_file(self, file_name, file_size):
        self.files.append(File(file_name, file_size, self.level+1))

    def print(self):
        print(f'{"  "*self.level}- {self.name} (dir, size={self.get_size()})')
        for subdir in self.subdirectories:
            subdir.print()
        for file in self.files:
            file.print()

class File():
    def __init__(self, name, size, level):
        self.name = name
        self.size = size
        self.level = level

    def get_size(self):
        return int(self.size)

    def print(self):
        print(f'{"  "*self.level}- {self.name} (file, size={self.size})')


def build_file_system(console):
    root = Directory('/', 0, None)
    active_directory = root

    for command in console:
        if command.startswith('$'):
            command = command.split(" ")[1:]
            if command[0] == 'cd':
                if command[1] == '/':
                    active_directory = root
                elif command[1] == '..':
                    active_directory = active_directory.parent
                else:
                    active_directory = active_directory.access_subdirectory(command[1])
        else:
            size, name = command.split(" ")
            if size == 'dir':
                active_directory.add_subdirectory(name)
            else:
                active_directory.add_file(name, size)
    
    print('Filesystem generated:')
    root.print()
    print()

    return root

def dirs_with_max_size(path, size):
    console = read_console(path)
    root = build_file_system(console)

    total_size = 0
    total_dirs = []
    queue = [root]

    while len(queue) > 0:
        dir = queue.pop(0)
        queue = queue + dir.subdirectories
        if dir.get_size() <= size:
            total_dirs.append(dir.name)
            total_size += dir.get_size()

    print(f'Directories with size less than {size}:')
    print(f'{total_dirs}')
    print()

    return total_size

def delete_smallest_dir(path, full_space, required_space):
    console = read_console(path)
    root = build_file_system(console)

    size_threshold = required_space - (full_space - root.get_size())

    dirs = []
    sizes = []
    queue = [root]

    while len(queue) > 0:
        dir = queue.pop(0)
        queue = queue + dir.subdirectories
        dir_size = dir.get_size()
        if dir_size >= size_threshold:
            dirs.append(dir.name)
            sizes.append(dir_size)

    min_size = min(sizes)
    min_dir = dirs[sizes.index(min_size)]

    return min_size, min_dir

In [95]:
example = "./example.txt"
test = "./test.txt"

max_size = 100000

example_max_size = dirs_with_max_size(example, max_size)
test_max_size = dirs_with_max_size(test, max_size)

print('##########################################################################################')
print('  *     *        *        *  *        *     *        *   *   *      *      *    *    *    ')
print(f'In the _example case_ the total size of folders with size less than {max_size} is {example_max_size}.')
print(f'In the _test case_ the total size of folders with size less than {max_size} is {test_max_size}.')
print('  *   *     *       *        *        *     *    *      *   *      *       *    *    *    ')
print('##########################################################################################')

Filesystem generated:
- / (dir, size=48381165)
  - a (dir, size=94853)
    - e (dir, size=584)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - d (dir, size=24933642)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)

Directories with size less than 100000:
['a', 'e']

Filesystem generated:
- / (dir, size=48748071)
  - cgw (dir, size=1994012)
    - jpmf (dir, size=1599361)
      - ncfcchsz (dir, size=205103)
        - ntpprbt.pvt (file, size=205103)
      - rfb (dir, size=331667)
        - tjtccqtm.qzv (file, size=331667)
      - tjtccqtm (dir, size=1062591)
        - jvbb (dir, size=255580)
          - fbhz (dir, size=255580)
            - wjj (dir, size=255580)
              - mgft (file, size=255580)
        - zgpb (dir, size=258152)
          - bgpm.gqf (file, size=57622)
          

In [104]:
full_space = 70000000
required_space = 30000000

example_deletion_size, example_deletion_dir = delete_smallest_dir(example, full_space, required_space)
test_deletion_size, test_deletion_dir = delete_smallest_dir(test, full_space, required_space)

print('##########################################################################################')
print('  *     *        *        *  *        *     *        *   *   *      *      *    *    *    ')
print(f'In the _example case_ the smallest deletable directory is {example_deletion_dir} (size={example_deletion_size}).')
print(f'In the _test case_ the smallest deletable directory is {test_deletion_dir} (size={test_deletion_size}).')
print('  *   *     *       *        *        *     *    *      *   *      *       *    *    *    ')
print('##########################################################################################')

Filesystem generated:
- / (dir, size=48381165)
  - a (dir, size=94853)
    - e (dir, size=584)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - d (dir, size=24933642)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)

Filesystem generated:
- / (dir, size=48748071)
  - cgw (dir, size=1994012)
    - jpmf (dir, size=1599361)
      - ncfcchsz (dir, size=205103)
        - ntpprbt.pvt (file, size=205103)
      - rfb (dir, size=331667)
        - tjtccqtm.qzv (file, size=331667)
      - tjtccqtm (dir, size=1062591)
        - jvbb (dir, size=255580)
          - fbhz (dir, size=255580)
            - wjj (dir, size=255580)
              - mgft (file, size=255580)
        - zgpb (dir, size=258152)
          - bgpm.gqf (file, size=57622)
          - jvbb (file, size=44482)
          - tpc (file, siz